In [1]:
import pandas as pd
from scipy.stats import entropy

In [2]:
from sklearn.datasets import load_iris
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.tree import DecisionTreeClassifier

In [3]:
# Задание 1
# Cоздайте Decision Tree классификатор, используя одноимённый класс из библиотеки sklearn и сохраните его в переменную dt.
# У дерева должны быть следующие параметры:
# - максимальная глубина - 5 уровней
# - минимальное число образцов в вершине для разделения - 5

In [4]:
dt = DecisionTreeClassifier(max_depth=5, min_samples_split=5)
dt

DecisionTreeClassifier(max_depth=5, min_samples_split=5)

In [5]:
# Задание 2
# Скачайте набор данных с тремя переменными: sex, exang, num.
# Представьте, что при помощи дерева решений мы хотим классифицировать есть или нет у пациента заболевание сердца
# (переменная num), основываясь на двух признаках: пол (sex) и наличие/отсутсвие стенокардии (exang).
# Обучите дерево решений на этих данных, используйте entropy в качестве критерия.
# Укажите, чему будет равняться значение Information Gain для переменной,  которая будет помещена в корень дерева.

In [6]:
task2_df = pd.read_csv("https://stepik.org/media/attachments/course/4852/train_data_tree.csv")
task2_df.head()

,sex,exang,num
0,1,1,1
1,1,1,1
2,1,0,1
3,1,0,0
4,1,0,1


In [7]:
# Разделим данные на признаки и целевую переменную
X = task2_df[["sex", "exang"]]  # Признаки
y = task2_df["num"]  # Целевая переменная

In [8]:
# Обучим дерево решений на полученных данных
dt = DecisionTreeClassifier(criterion="entropy", max_depth=5, min_samples_split=5)
dt.fit(X, y)

DecisionTreeClassifier(criterion='entropy', max_depth=5, min_samples_split=5)

In [9]:
# Для вычисления Information Gain, нам необходимо посмотреть на разделение в корне дерева
# Это можно сделать, изучив важность признаков в модели
feature_importances = dt.feature_importances_
feature_importances

array([0.24090887, 0.75909113])

In [10]:
# Рассчитаем энтропию целевой переменной до разделения:
# Для этого подсчитаем количества каждого класса в целевой переменной и вероятность
num_counts = y.value_counts()
num_probs = num_counts / len(y)

# Вычислим энтропию для целевой переменной
initial_entropy = entropy(num_probs, base=2)
initial_entropy

0.9958699965528488

In [11]:
# Разделяем данные на основе значения переменной exang
exang_groups = task2_df.groupby("exang")

# Инициализируем переменную для суммирования средневзвешенной энтропии после разделения
weighted_entropy_after_split = 0

for name, group in exang_groups:
    # Подсчет количества каждого класса в группе
    num_counts = group["num"].value_counts()
    # Вычисление вероятностей для каждого класса в группе
    num_probs = num_counts / len(group)
    # Вычисление энтропии для каждой группы
    group_entropy = entropy(num_probs, base=2)
    # Добавление взвешенной энтропии группы к общей сумме
    weighted_entropy_after_split += group_entropy * (len(group) / len(task2_df))

# Вычислим Information Gain (IG)
information_gain = initial_entropy - weighted_entropy_after_split
round(information_gain, 3)

0.119

In [12]:
# Задание 3
# Теперь, создав дерево, давайте обучим его и попробуем что-нибудь предсказать!
# Для начала опробуем наше дерево на классическом наборе iris,
# где собраны данные о длине, ширине чашелистиков и лепестков ирисов и их принадлежности к виду.
# В sklearn он уже встроен, что довольно удобно.
# Итак, вам даны 2 numpy эррея с измеренными признаками ирисов и их принадлежностью к виду.
# Сначала попробуем примитивный способ с разбиением данных на 2 датасэта.
# Используйте функцию train_test_split для разделения имеющихся данных на тренировочный и тестовый наборы данных,
# 75% и 25% соответственно.
# Затем создайте дерево dt с параметрами по умолчанию и обучите его на тренировочных данных,
# а после предскажите классы, к которым принадлежат данные из тестовой выборки,
# сохраните результат предсказаний в переменную predicted.

In [13]:
# Загружаем dataset iris и разденяем его на признаки (X) и целевые значения (y)
iris = load_iris()
X, y = iris.data, iris.target

In [14]:
# Разделим данные на тренировочные и тестовые наборы
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [15]:
# Создаем и обучаем дерево решений
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)

DecisionTreeClassifier()

In [16]:
# Предсказываем классы для тестовых данных
predicted = dt.predict(X_test)

In [17]:
# Задание 4
# Одно дерево - хорошо, но где гарантии, что оно является лучшим, или хотя бы близко к нему?
# Одним из способов найти более-менее оптимальный набор параметров дерева
# является перебор множества деревьев с разными параметрами и выбор подходящего.
# Для этой цели существует класс GridSearchCV, перебирающий каждое из сочетаний параметров
# среди заданных для модели, обучающий её на данных и проводящих кросс-валидацию.
# После этого в аттрибуте .best_estimator_ храниться модель с лучшими параметрами.
# Это применимо не только к деревьям, но и к другим моделям sklearn.

# Теперь задание - осуществите перебор всех деревьев на данных ириса по следующим параметрам:
# - максимальная глубина - от 1 до 10 уровней
# - минимальное число проб для разделения - от 2 до 10
# - минимальное число проб в листе - от 1 до 10
# и сохраните в переменную best_tree лучшее дерево. Переменную с GridSearchCV назовите search

In [18]:
# Загружаем dataset iris и разденяем его на признаки (X) и целевые значения (y)
iris = load_iris()
X, y = iris.data, iris.target

# Задаем параметры для перебора
param_grid = {"max_depth": range(1, 11), "min_samples_split": range(2, 11), "min_samples_leaf": range(1, 11)}

# Инициализируем классификатор и обучаем GridSearchCV
dt = DecisionTreeClassifier()
search = GridSearchCV(dt, param_grid, cv=5)
search.fit(X, y)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': range(1, 11),
                         'min_samples_leaf': range(1, 11),
                         'min_samples_split': range(2, 11)})

In [19]:
# Сохраняем лучшее дерево
best_tree = search.best_estimator_
best_tree

DecisionTreeClassifier(max_depth=3, min_samples_split=4)

In [20]:
# Задание 5
# Чем больше данных, сложность модели и число её параметров, тем дольше будет вестись поиск GridSearchCV.
# Однако бывают случаи, когда модель нужна здесь и сейчас, и для этого есть RandomizedSearchCV!
# Пробегаясь по рандомной подвыборке параметров, он ищет наиболее хорошую модель
# и делает это быстрее полного перебора параметров, хотя и может пропустить оптимальные параметры.
# Здесь можно посмотреть на сравнение этих поисков.
# Осуществим поиск по тем же параметрам что и в предыдущем задании с помощью RandomizedSearchCV
# - максимальная глубина - от 1 до 10 уровней
# - минимальное число проб для разделения - от 2 до 10
# - минимальное число проб в листе - от 1 до 10
# Cохраните в переменную best_tree лучшее дерево. Переменную с RandomizedSearchCV назовите search

In [21]:
# Загружаем dataset iris и разденяем его на признаки (X) и целевые значения (y)
iris = load_iris()
X, y = iris.data, iris.target

# Задаем параметры для перебора
param_distributions = {"max_depth": range(1, 11), "min_samples_split": range(2, 11), "min_samples_leaf": range(1, 11)}

# Инициализируем классификатор и RandomizedSearchCV
# n_iter установлен на 100, чтобы просмотреть достаточное количество комбинаций,
# но можно установить меньшее значение для более быстрого поиска
dt = DecisionTreeClassifier()
search = RandomizedSearchCV(dt, param_distributions, n_iter=10, cv=5, random_state=42)

# Обучаем RandomizedSearchCV
search.fit(X, y)

RandomizedSearchCV(cv=5, estimator=DecisionTreeClassifier(),
                   param_distributions={'max_depth': range(1, 11),
                                        'min_samples_leaf': range(1, 11),
                                        'min_samples_split': range(2, 11)},
                   random_state=42)

In [22]:
# Сохраняем лучшее дерево
best_tree = search.best_estimator_
best_tree

DecisionTreeClassifier(max_depth=9, min_samples_leaf=2, min_samples_split=4)

In [23]:
# Задание 6
# Воспользуемся изученными приёмами и попредсказываем!
# Даны 2 датасэта, к которым вы можете обращаться:
# - train - размеченный с известными правильным ответами (хранятся в колонке y)
# - test - набор, где нужно предсказать их
# Найдите дерево с наиболее подходящими параметрами с помощью GridSearchCV
# и предскажите с его помощью ответы ко 2-ому сэту! Границы параметров как раньше:
# - максимальная глубина - от 1 до 10 уровней
# - минимальное число проб для разделения - от 2 до 10
# - минимальное число проб в листе - от 1 до 10
# Названия переменных тоже:лучшее дерево - best_tree, GridSearchCV - search, а предсказания - predictions

In [24]:
# Предполагается, что переменные train и test уже определены и загружены заранее
# Для создания имитационных тренировочных и тестовых датасетов используем датасет iris

# Загружаем dataset iris и разденяем его на признаки (X) и целевые значения (y)
iris = load_iris()
X, y = iris.data, iris.target

# Для имитации мы можем сделать первые 75% данных тренировочными, а оставшиеся 25% - тестовыми.
train_size = int(0.75 * X.shape[0])

# Создаем имитационные тренировочные и тестовые наборы
train = pd.DataFrame(X[:train_size, :], columns=iris.feature_names)
train["y"] = y[:train_size]
test = pd.DataFrame(X[train_size:, :], columns=iris.feature_names)

In [25]:
# Разделяем тренировочный датасет на признаки и целевую переменную
X_train = train.drop("y", axis=1)
y_train = train["y"]

# Создаем параметры для GridSearchCV
param_grid = {"max_depth": range(1, 11), "min_samples_split": range(2, 11), "min_samples_leaf": range(1, 11)}

# Инициализируем DecisionTreeClassifier и GridSearchCV и обучаем на тренировочных данных
dt = DecisionTreeClassifier()
search = GridSearchCV(dt, param_grid, cv=5)
search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': range(1, 11),
                         'min_samples_leaf': range(1, 11),
                         'min_samples_split': range(2, 11)})

In [26]:
# Сохраняем лучшее дерево
best_tree = search.best_estimator_
best_tree

DecisionTreeClassifier(max_depth=2, min_samples_leaf=10)

In [27]:
# Делаем предсказания на тестовых данных
predictions = best_tree.predict(test)
predictions

array([2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1,
       1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [28]:
# Задание 6
# При классификации модель может допускать ошибки, присваивая наблюдению неверный класс.
# Существуют различные метрики оценки качества предсказаний, которые базируются на 4-х параметрах:
# true positive, false positive, false negative и true negative,
# соответствующих тому какой класс был присвоен наблюдениям каждого из классов.
# Матрицу из 4-ёх (в случае бинарной классификации) этих параметров называют confusion matrix.

# В sklearn можно её удобно получить с помощью функции confusion_matrix.
# Вам даны 2 эррея с истинными классами наблюдений и предсказанными - y и predictions.
# Получите по ним confusion matrix и поместите её в переменную conf_matrix.

In [29]:
# Предполагается, что переменные y и predictions уже определены и загружены заранее
# Симулируем эти значения для проверки решения
y = [1, 0, 1, 1, 0, 1, 0, 0, 1, 1]  # Истинные классы
predictions = [1, 0, 1, 1, 0, 0, 0, 0, 1, 1]  # Предсказанные классы

conf_matrix = confusion_matrix(y, predictions)
conf_matrix

array([[4, 0],
       [1, 5]])